In [1]:
import glob
import os
import librosa
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
%matplotlib inline
plt.style.use('ggplot')

/home/tianwei/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [3]:
#### Logit to evidence converters - activation functions (they have to produce non-negative outputs for the uncertaintyuncertainity process)

def relu_evidence(logits):
    return tf.nn.relu(logits)

def exp_evidence(logits):                  # tune the parameters  1000
    return tf.exp(logits/1)

def softplus(logits):
    return tf.keras.activations.softplus(logits)
  
def relu6_evidence(logits):
    return tf.nn.relu6(logits)
  
def softsign_evidence(logits):
    return tf.nn.softsign(logits)

  
#### KL Divergence calculator

def KL(alpha, K):
    beta=tf.constant(np.ones((1,K)),dtype=tf.float32)
    S_alpha = tf.reduce_sum(alpha,axis=1,keepdims=True)
    
    KL = tf.reduce_sum((alpha - beta)*(tf.digamma(alpha)-tf.digamma(S_alpha)),axis=1,keepdims=True) + \
         tf.lgamma(S_alpha) - tf.reduce_sum(tf.lgamma(alpha),axis=1,keepdims=True) + \
         tf.reduce_sum(tf.lgamma(beta),axis=1,keepdims=True) - tf.lgamma(tf.reduce_sum(beta,axis=1,keepdims=True))
    return KL


##### Loss functions (there are three different one defined in the papaer)

def loss_eq5(p, alpha, K, global_step, annealing_step):  # MSE
    S = tf.reduce_sum(alpha, axis=1, keepdims=True)
    loglikelihood = tf.reduce_sum((p-(alpha/S))**2, axis=1, keepdims=True) + tf.reduce_sum(alpha*(S-alpha)/(S*S*(S+1)), axis=1, keepdims=True)
    KL_reg =  tf.minimum(1.0, tf.cast(global_step/annealing_step, tf.float32)) * KL((alpha - 1)*(1-p) + 1 , K)
    return loglikelihood + KL_reg

def loss_eq4(p, alpha, K, global_step, annealing_step):   #  expected cross entropy loss
    loglikelihood = tf.reduce_mean(tf.reduce_sum(p * (tf.digamma(tf.reduce_sum(alpha, axis=1, keepdims=True)) - tf.digamma(alpha)), 1, keepdims=True))
    KL_reg =  tf.minimum(1.0, tf.cast(global_step/annealing_step, tf.float32)) * KL((alpha - 1)*(1-p) + 1 , K)
    return loglikelihood + KL_reg

def loss_eq3(p, alpha, K, global_step, annealing_step):
    loglikelihood = tf.reduce_mean(tf.reduce_sum(p * (tf.log(tf.reduce_sum(alpha, axis=1, keepdims=True)) - tf.log(alpha)), 1, keepdims=True))
    KL_reg =  tf.minimum(1.0, tf.cast(global_step/annealing_step, tf.float32)) * KL((alpha - 1)*(1-p) + 1 , K)
    return loglikelihood + KL_reg

def mse_loss(p, alpha, K, global_step, annealing_step): 
    S = tf.reduce_sum(alpha, axis=1, keep_dims=True) 
    E = alpha - 1
    m = alpha / S
    
    A = tf.reduce_sum((p-m)**2, axis=1, keep_dims=True) 
    B = tf.reduce_sum(alpha*(S-alpha)/(S*S*(S+1)), axis=1, keep_dims=True) 
    
    annealing_coef = tf.minimum(1.0,tf.cast(global_step/annealing_step,tf.float32))
    
    alp = E*(1-p) + 1 
    C =  annealing_coef * KL(alp)
    return (A + B) + C


In [4]:
def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

# Load data

In [5]:

## GET DATA TO WORK ON
print("Start loading data")

fd = open("data_x.pkl", 'rb')
fd2 = open("data_y.pkl", 'rb')
features = pickle.load(fd)
labels = pickle.load(fd2)

print("Data loaded")

# TRAIN - TEST
p_train = 0.8

rnd_indices = np.random.rand(len(labels)) < p_train
X_train = features[rnd_indices]
Y_train = labels[rnd_indices]
X_test = features[~rnd_indices]
Y_test = labels[~rnd_indices]

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255


## FIX FOR KERAS
# Y_train = Y_train.reshape((-1, 1))
# Y_test = Y_test.reshape((-1, 1))

## one hot encoding
Y_train = one_hot_encode(Y_train)
Y_test = one_hot_encode(Y_test)

Start loading data
Data loaded


In [6]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(43258, 40, 40, 3)
(43258, 10)
(10896, 40, 40, 3)
(10896, 10)


In [7]:
# NETWORK PARAMETERS
data_w = 40
data_h = 40
n_classes = 10
n_filters_1 = 32
n_filters_2 = 64
d_filter = 3
p_drop_1 = 0.25
p_drop_2 = 0.50


batch_size = 32
nb_epoch = 20


K= n_classes
num_channels = 3
num_labels = n_classes

n_batches = Y_train.shape[0]//batch_size
n_batches_test = Y_test.shape[0]//batch_size
print('For batch of size %d: \n %d batches in train \n %d batches in test'%(batch_size, n_batches, n_batches_test))




For batch of size 32: 
 1351 batches in train 
 340 batches in test


In [9]:
lmb = 0.1
omega = 1.0
regularizer = tf.contrib.layers.l2_regularizer(scale=omega)

In [10]:
# new network:

X = tf.placeholder(tf.float32, shape=[None,data_w,data_h,num_channels])
Y = tf.placeholder(tf.float32, shape=[None,num_labels])

keep_prob = tf.placeholder(dtype=tf.float32)
global_step = tf.Variable(initial_value=0, name='global_step', trainable=False)
annealing_step = tf.placeholder(dtype=tf.int32) 

### conv module

# Convolutional Layer #1
conv1 = tf.layers.conv2d(
    inputs=X,
    filters=32,
    strides=(1, 1),
    kernel_size=[3, 3],
    kernel_regularizer=regularizer,
    padding="valid"
    )
conv1_act = tf.nn.relu( conv1 )
# pool1 = tf.layers.max_pooling2d(inputs=act1, pool_size=[3, 3], strides=3)
# dropout1 = tf.layers.dropout(
#     inputs=pool1, rate=0.1)

# Convolutional Layer #2
conv2 = tf.layers.conv2d(
    inputs=conv1_act,
    filters=32,
    strides=(1, 1),
    kernel_size=[3, 3],
    kernel_regularizer=regularizer,
    padding="valid"
    )
# bn2 = tf.layers.batch_normalization(
#     conv2,
#     axis=-1
#     )
conv2_act = tf.nn.relu( conv2 )
conv2_mp = tf.layers.max_pooling2d(inputs=conv2_act, pool_size=[2, 2], strides=2)
dpout1 = tf.layers.dropout(
    inputs=conv2_mp, rate= p_drop_1)



# Convolutional Layer #3
conv3 = tf.layers.conv2d(
    inputs=dpout1,
    filters=64,
    strides=(1, 1),
    kernel_size=[3, 3],
    kernel_regularizer=regularizer,
    padding="valid"
    )
conv3_act = tf.nn.relu( conv3 )

# Convolutional Layer #4
conv4 = tf.layers.conv2d(
    inputs=conv1_act,
    filters=64,
    strides=(1, 1),
    kernel_size=[3, 3],
    kernel_regularizer=regularizer,
    padding="valid"
    )
# bn2 = tf.layers.batch_normalization(
#     conv2,
#     axis=-1
#     )
conv4_act = tf.nn.relu( conv4 )
conv4_mp = tf.layers.max_pooling2d(inputs=conv4_act, pool_size=[2, 2], strides=2)
dpout2 = tf.layers.dropout(
    inputs=conv4_mp, rate= p_drop_1)





### modify dimensions
shape = dpout2.get_shape().as_list()
flat1 = tf.reshape(dpout2, [-1, shape[1] * shape[2]* shape[3]])



### dense module

fc1 = tf.layers.dense(inputs=flat1, 
                          kernel_regularizer=regularizer,
                          units=256)
fc1_act = tf.nn.relu( fc1 )
dpout3 = tf.layers.dropout(
    inputs=fc1_act, rate= p_drop_2)

# Logits Layer
logits = tf.layers.dense(inputs=dpout3, 
                         kernel_regularizer=regularizer,
                         units=n_classes)


y_ = tf.nn.softmax(logits,name="softmax_tensor")


prediction = tf.argmax(logits, 1)



########### EDL extension ###########
 
logits2evidence = relu_evidence  ############ modify this function:   exp_evidence softplus

evidence = logits2evidence(logits)
alpha = evidence + 1

u = K / tf.reduce_sum(alpha, axis=1, keepdims=True)

prob = alpha/tf.reduce_sum(alpha, 1, keepdims=True) 

loss_function = loss_eq5  ########### use 5th MSE loss equ: loss_eq5, loss_eq4, loss_eq3, mse_loss

loss = tf.reduce_mean(loss_function(Y, alpha, K, global_step, annealing_step))
l2_loss = tf.losses.get_regularization_loss() * lmb
loss_func = loss + l2_loss

optimizer = tf.train.AdamOptimizer().minimize(loss_func, global_step=global_step)

match = tf.reshape(tf.cast(tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1)), tf.float32),(-1,1))
accuracy = tf.reduce_mean(match)

In [11]:
learning_rate = 0.01
training_iterations = 100

In [12]:
# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
# l2_loss = tf.losses.get_regularization_loss()
# loss_func = loss + l2_loss*0


# # optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy)
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss_func)

# correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(Y,1))
# accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
cost_train = []
cost_test = []
acc_train = []
acc_test = []

session =  tf.Session(config=config)
session.run(tf.global_variables_initializer())

# with tf.Session(config=config) as session:
#     tf.global_variables_initializer().run()

for itr in range(training_iterations):  
    acc_list = []
    loss_list = []
    for i in range(n_batches):
        offset = (i * batch_size) % (Y_train.shape[0] - batch_size)
        batch_x = X_train[offset:(offset + batch_size), :, :, :]
        batch_y = Y_train[offset:(offset + batch_size), :]

        _, c, acc = session.run([optimizer, loss_func, accuracy],feed_dict={X: batch_x, Y : batch_y, keep_prob:.5, annealing_step:100*n_batches})
        print('epoch %d - %d%% -%f) '% (itr+1, (100*(i+1))//n_batches, acc), end='\r' if i<n_batches-1 else '')
        acc_list.append(acc)
        loss_list.append(c)
    train_acc = np.array(np.array(acc_list).mean())
    train_loss = np.array(np.array(loss_list).sum())
    print('Training Loss: %2.4f \t Accuracy: %2.4f' % (train_loss/Y_train.shape[0], train_acc))


#       Performance on testing dataset:
    acc_list = []
    loss_list = []
    for i in range(n_batches_test):
        offset = (i * batch_size) % (Y_test.shape[0] - batch_size)
        batch_x = X_test[offset:(offset + batch_size), :, :, :]
        batch_y = Y_test[offset:(offset + batch_size), :]

        c, acc = session.run([loss_func, accuracy],feed_dict={X: batch_x, Y : batch_y, keep_prob:1.,  annealing_step:100*n_batches})
#             print('epoch %d - %d%% -%f) '% (itr+1, (100*(i+1))//n_batches, c), end='\r' if i<n_batches-1 else '')
        acc_list.append(acc)
        loss_list.append(c)
    test_acc = np.array(np.array(acc_list).mean())
    test_loss = np.array(np.array(loss_list).sum())
    print('Testing:\t  Loss: %2.4f \t Accuracy: %2.4f' % (test_loss/Y_test.shape[0], test_acc))

    cost_train.append(train_loss)
    cost_test.append(test_loss)
    acc_train.append(train_acc)
    acc_test.append(test_acc)

epoch 1 - 100% -0.000000) Training Loss: 0.0383 	 Accuracy: 0.1310
Testing:	  Loss: 0.0305 	 Accuracy: 0.1165
epoch 2 - 100% -0.000000) Training Loss: 0.0306 	 Accuracy: 0.1340
Testing:	  Loss: 0.0307 	 Accuracy: 0.1165
epoch 3 - 100% -0.000000) Training Loss: 0.0305 	 Accuracy: 0.1291
Testing:	  Loss: 0.0304 	 Accuracy: 0.1165
epoch 4 - 100% -0.000000) Training Loss: 0.0303 	 Accuracy: 0.1222
Testing:	  Loss: 0.0305 	 Accuracy: 0.1165
epoch 5 - 100% -0.000000) Training Loss: 0.0304 	 Accuracy: 0.1234
Testing:	  Loss: 0.0305 	 Accuracy: 0.1165
epoch 6 - 100% -0.000000) Training Loss: 0.0304 	 Accuracy: 0.1246
Testing:	  Loss: 0.0305 	 Accuracy: 0.1165
epoch 7 - 100% -0.000000) Training Loss: 0.0304 	 Accuracy: 0.1233
Testing:	  Loss: 0.0305 	 Accuracy: 0.1165
epoch 8 - 100% -0.000000) Training Loss: 0.0305 	 Accuracy: 0.1229
Testing:	  Loss: 0.0305 	 Accuracy: 0.1165
epoch 9 - 100% -0.000000) Training Loss: 0.0305 	 Accuracy: 0.1211
Testing:	  Loss: 0.0305 	 Accuracy: 0.1165
epoch 10 -

In [ ]:
# cost_train
# cost_test
# acc_train
# acc_test
fig = plt.figure(figsize=(9,3))
plt.plot(cost_train, 'r-')
plt.plot(cost_test, 'b-')

# plt.axis([0,training_iterations,0,np.max(cost_history)])
plt.show()


fig = plt.figure(figsize=(9,3))
plt.plot(acc_train, 'r-')
plt.plot(acc_test, 'b-')
# plt.axis([0,training_iterations,0,np.max(cost_history)])
plt.show()

# Evaluate model

In [ ]:
pred_y_list = np.zeros(0)

acc_list = []
loss_list = []

#       Performance on testing dataset:
for i in range(n_batches_test):
    if i == n_batches_test-1:
        batch_x = X_test[i * batch_size:, :, :, :]
        batch_y = Y_test[i * batch_size:, :]
    else:
        offset = (i * batch_size) % (Y_test.shape[0] - batch_size)
        batch_x = X_test[offset:(offset + batch_size), :, :, :]
        batch_y = Y_test[offset:(offset + batch_size), :]

#     logits, y_pred = session.run([logits, prediction], feed_dict={X: batch_x, Y : batch_y})
    y_pred, acc, c = session.run([prediction, accuracy, loss_func], feed_dict={X: batch_x, Y : batch_y})

    print('epoch %d - %d%% -%f) '% (i+1, (100*(i+1))//n_batches_test, acc), end='\r' if i<n_batches_test-1 else '')
#     y_pred = np.argmax(logits, axis=1)
#     pred_y_list.append(y_pred)
    pred_y_list = np.concatenate([pred_y_list, y_pred])
    
    acc_list.append(acc)
    loss_list.append(c)
#     test_acc = np.array(np.array(acc_list).mean())
#     test_loss = np.array(np.array(loss_list).sum())
    

test_acc = np.array(np.array(acc_list).mean())
test_loss = np.array(np.array(loss_list).sum())
print('Testing:\t  Loss: %2.4f \t Accuracy: %2.4f' % (test_loss/Y_test.shape[0], test_acc))

In [ ]:
y_true = np.argmax(Y_test, 1)
y_pred =pred_y_list

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score



print('Accuracy on testing data:',sum(y_pred==y_true)/y_true.shape[0])
cf_matrix = confusion_matrix(y_true, y_pred)
print(cf_matrix)
class_wise_f1 = np.round(f1_score(y_true, y_pred, average=None)*100)*0.01
print('the mean-f1 score: {:.2f}'.format(np.mean(class_wise_f1)))

In [ ]:
Y_test